Spending data processing

In this section I will process the data from the FEC to get the total amount spent by each presidential candidate in each state for the 2008, 2012, 2016, 2020, and 2024 elctions. 

Raw data is from here: https://www.fec.gov/data/candidates/president/presidential-map/ 
To get the data yourself, use the dropdown menu to select the electin year, open the tab on the side of the map for spending, and click "Export spending data". 
The files do not have labeled names so you need to label them accordingly, we have followed the pattern "spending_data_{year}.csv". 

In [261]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import scipy
from scipy.stats import wilcoxon
import datetime

Import spending data of presidential candidates for the 2008, 2012, 2016, 2020, and 2024 elctions. 

In [262]:
years = ['2008', '2012', '2016', '2020', '2024']
spending_dfs = dict()
state_sums = dict()
for year in years:
    spending_dfs[year] = pd.read_csv(f"spending_data_{year}.csv", index_col=False, low_memory=False)
    # state_sums[year] = spending_dfs[year].groupby('recipient_st').sum('disb_amt').sort_values('disb_amt')

# spending_24_df = pd.read_csv("spending_data_2024.csv", index_col=False, low_memory=False)
# spending_24_df.groupby('recipient_st').sum('disb_amt').sort_values('disb_amt')
# spending_24_df.head()
# spending_dfs
# state_sums['2024']

Upon inspection some states have been labeled wrong. For example some rows have C, since they occur in san francisco we can assume this should be CA. 

For each year I have a comment labeling which abbreviations need to be replaced, for example in 2008 'AA' needs to be replaced by 'MA'. 

Most were done with a simple dictionary replace, but in 2024 some bad labels were for different states, for example zip codes in Iowa and Indiana were both labled 'I', so these require extra attention to replace them by zip code. 

Then once all errors are corrected we remove all rows where the recipient state is not one of the 50 US states or DC, for example some transactions are labled as "UK" which isn't useful for our analysis. 

In [263]:
# correct 2008 states 
spending_dfs['2008']['recipient_st']= spending_dfs['2008']['recipient_st'].replace({'AA': 'MA', 'C': 'CA', 'I': 'IA', 'II': 'IL', 'K': 'KS', 'KA': 'KS' , 'N': 'NC', 'VW': 'WV', '46': 'IN', 'MY': 'MT', 'OA': 'PA', 'T': 'TX', 'WW': 'WA'})

# AA = MA
# C = CA
# I = IA
# II = IL
# K = KS
# KA = KS 
# N = NC
# VW = WV
# 46 = IN
# MY = MT
# OA = PA
# T = TX
# WW = WA


# correct 2012 states 
spending_dfs['2012']['recipient_st']= spending_dfs['2012']['recipient_st'].replace({'D.': 'DC', 'MY': 'NY', 'HA': 'HI', 'MH': 'NH', 'HN': 'NH'})

# "D." = "DC"
# MY = NY
# HA = HI
# MH = NH
# HN = HN

# correct 2016 states 
spending_dfs['2016']['recipient_st']= spending_dfs['2016']['recipient_st'].replace({'D.': 'DC', 'MY': 'NY', 'HA': 'HI', 'MH': 'NH', 'HN': 'NH'})

# NB = NE
# D. = DC


# correct 2020 states 
# none


# correct 2024 states 
spending_dfs['2024']['recipient_st']= spending_dfs['2024']['recipient_st'].replace({'C': 'CA', 'AA': 'CA', 'F': 'FL', 'G': 'GA', 'T': 'TX'})
spending_dfs['2024'].loc[spending_dfs['2024']['recipient_zip'] == '46038', 'recipient_st'] = 'IN'
spending_dfs['2024']['recipient_st'] = spending_dfs['2024']['recipient_st'].replace('I', 'IA')
spending_dfs['2024'].loc[spending_dfs['2024']['recipient_zip'] == '03276', 'recipient_st'] = 'NH'
spending_dfs['2024'].loc[spending_dfs['2024']['recipient_zip'] == '03063', 'recipient_st'] = 'NH'
spending_dfs['2024']['recipient_st'] = spending_dfs['2024']['recipient_st'].replace('N', 'NY')
spending_dfs['2024'].loc[spending_dfs['2024']['recipient_zip'] == '19802', 'recipient_st'] = 'NJ'

# C = CA,
# AA = CA
# F = FL
# G = GA
# T = TX
# I = IN or IA
# N = NH or NY


states = [
        'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'IA', 
        'IL', 'IN', 'LA', 'MA', 'MD', 'ME', 'MN', 'MO', 'NC', 'NE', 
        'NH', 'NJ', 'NY', 'OK', 'PA', 'SC', 'TN', 'TX', 'VA', 'SD',
        'WA', 'WI', 'WY', 'OH', 'WV', 'AK', 'DE', 'HI', 'ID', 'KS', 
        'KY', 'MI', 'MS', 'MT', 'ND', 'NM', 'NV', 'OR', 'RI', 'UT', 
        'VT'
        ]

# remove transactions not in the 50 states or DC
for year in spending_dfs:
    spending_dfs[year] = spending_dfs[year][spending_dfs[year]['recipient_st'].isin(states)]


Next we want to remove the candidates that aren't either the Democratic or Republican nominee, while other candidates exist they don't receive a significant amount of votes. This will leave us with a dataframe per candidate per election year, so 10 in total. 

In [264]:
party_spending_dfs = {}

# 2008
# Republican: 'McCain, John S'
# Dem : 'Obama, Barack'
# spending_dfs['2008'] = spending_dfs['2024'][spending_dfs['2024']['cand_nm'].isin(['McCain, John S', 'Obama, Barack'])]
party_spending_dfs['2008_R'] = spending_dfs['2008'][spending_dfs['2008']['cand_nm'] == 'McCain, John S']
party_spending_dfs['2008_D'] = spending_dfs['2008'][spending_dfs['2008']['cand_nm'] == 'Obama, Barack']

# 2012
# Rep: 'Romney, Mitt'
# Dem: 'Obama, Barack'
# spending_dfs['2012'] = spending_dfs['2024'][spending_dfs['2024']['cand_nm'].isin(['Romney, Mitt', 'Obama, Barack'])]
party_spending_dfs['2012_R'] = spending_dfs['2012'][spending_dfs['2012']['cand_nm'] == 'Romney, Mitt']
party_spending_dfs['2012_D'] = spending_dfs['2012'][spending_dfs['2012']['cand_nm'] == 'Obama, Barack']

# 2016
# Rep: 'Trump, Donald J.'
# Dem: 'Clinton, Hillary Rodham'
# spending_dfs['2016'] = spending_dfs['2024'][spending_dfs['2024']['cand_nm'].isin(['Trump, Donald J.', 'Clinton, Hillary Rodham'])]
party_spending_dfs['2016_R'] = spending_dfs['2016'][spending_dfs['2016']['cand_nm'] == 'Trump, Donald J.']
party_spending_dfs['2016_D'] = spending_dfs['2016'][spending_dfs['2016']['cand_nm'] == 'Clinton, Hillary Rodham']

# 2020
# Rep: 'Trump, Donald J.'
# Dem: 'Biden, Joseph R Jr'
# spending_dfs['2020'] = spending_dfs['2024'][spending_dfs['2024']['cand_nm'].isin(['Trump, Donald J.', 'Biden, Joseph R Jr'])]
party_spending_dfs['2020_R'] = spending_dfs['2020'][spending_dfs['2020']['cand_nm'] == 'Trump, Donald J.']
party_spending_dfs['2020_D'] = spending_dfs['2020'][spending_dfs['2020']['cand_nm'] == 'Biden, Joseph R Jr']

# 2024
# Rep: 'Trump, Donald J.'
# Dem: 'Harris, Kamala' 
# spending_dfs['2024'] = spending_dfs['2024'][spending_dfs['2024']['cand_nm'].isin(['Trump, Donald J.', 'Harris, Kamala'])]
party_spending_dfs['2024_R'] = spending_dfs['2024'][spending_dfs['2024']['cand_nm'] == 'Trump, Donald J.']
party_spending_dfs['2024_D'] = spending_dfs['2024'][spending_dfs['2024']['cand_nm'] == 'Harris, Kamala']


Nebraska and Maine split their electoral votes, 1 vote goes to the popular winner in each congressional district while 2 votes go to the popular winner of the whole state. So we will duplicate spending in Nebraska and Maine to count towards the overall state and the district it happened in, since later these will be predicted seperatley. 

In [265]:
# for year, df in party_spending_dfs.items():
#     print(year, df[df['recipient_st'].isin(['NE', 'ME'])]['recipient_city'].unique())

year = '2008'
print(f'{year}_R', party_spending_dfs[f'{year}_R'][party_spending_dfs[f'{year}_R']['recipient_st'] == 'NE']['recipient_city'].unique())
print(f'{year}_D', party_spending_dfs[f'{year}_D'][party_spending_dfs[f'{year}_D']['recipient_st'] == 'NE']['recipient_city'].unique())

print(f'{year}_R', party_spending_dfs[f'{year}_R'][party_spending_dfs[f'{year}_R']['recipient_st'] == 'ME']['recipient_city'].unique())
print(f'{year}_D', party_spending_dfs[f'{year}_D'][party_spending_dfs[f'{year}_D']['recipient_st'] == 'ME']['recipient_city'].unique())


2008_R ['OMAHA' 'NORFOLK' 'SOUTH SIOUX CITY']
2008_D ['ELM CREEK' 'OMAHA' 'YORK' 'NORTH PLATTE' '800-2289872' 'S. SIOUX CITY'
 'LINCOLN' 'SEWARD' 'KIMBALL' 'GRAND ISLAND' 'WACO' 'AURORA' 'KEARNEY'
 'SUTHERLAND' 'N. PLATTE' 'OGALLALA' 'NEW YORK' 'SIDNEY' 'NORFOLK'
 'LEXINGTON' 'COZAD' 'GERING' 'SOUTH SIOUX CITY' 'WINNEBAGO' 'LA VISTA'
 'MCCOOK' 'SUITE 270']
2008_R ['GRAY' 'S PORTLAND' 'AUGUSTA' 'SOUTH PORTLAND' 'PORTLAND' 'SCARBOROUGH'
 'KENNEBUNKPORT' 'BREWER' 'BANGOR' 'PRESQUE ISLE' 'ORONO' 'AUBURN'
 'PALMYRA' 'CORRINA']
2008_D ['SCARBOROUGH' 'CAPE ELIZABETH' 'BIDDEFORD' 'PORTLAND' 'BRUNSWICK'
 'CHINA VILLAGE' 'KITTERY' 'WINTHROP' 'BELFAST' 'ORONO' 'LEWISTON'
 'CAPE NEDDICK' 'WATERVILLE' 'AUGUSTA' 'BANGOR' 'SACO' 'SOUTH PORTLAND'
 'KENNEBUNK' 'WOOLWICH' 'CAMDEN' 'HAMPTDEN' 'LINCOLN' 'PRESQUE ISLE']


In [266]:
# 2008 
NE_1_2008 = ['NORFOLK', 'SOUTH SIOUX CITY', 'S. SIOUX CITY', 'SEWARD', 'NORFOLK', 'WINNEBAGO', 'LINCOLN']
NE_2_2008 = ['OMAHA', '800-2289872', 'NEW YORK', 'LA VISTA', 'SUITE 270']
NE_3_2008 = ['ELM CREEK', 'YORK', 'NORTH PLATTE', 'KIMBALL', 'GRAND ISLAND', 'WACO', 'AURORA','KEARNEY', 
             'SUTHERLAND', 'N. PLATTE', 'OGALLALA', 'SIDNEY', 'LEXINGTON', 'COZAD', 'GERING', 'MCCOOK']

ME_1_2008 = ['GRAY', 'S PORTLAND', 'AUGUSTA', 'SOUTH PORTLAND', 'PORTLAND', 'SCARBOROUGH', 'KENNEBUNKPORT', 
             'CAPE ELIZABETH', 'BIDDEFORD', 'BRUNSWICK', 'CHINA VILLAGE', 'KITTERY', 'WINTHROP', 'LEWISTON', 
             'CAPE NEDDICK', 'SACO', 'KENNEBUNK', 'WOOLWICH', 'CAMDEN']
ME_2_2008 = ['BREWER', 'BANGOR', 'PRESQUE ISLE', 'ORONO', 'AUBURN', 'PALMYRA', 'CORRINA', 'BELFAST',
             'WATERVILLE', 'HAMPTDEN']

# 2012
# Nebraksa used the same districts from 2003-2013
NE_1_2012 = ['SOUTH SIOUX CITY', 'LINCOLN']
NE_2_2012 = ['OMAHA', 'LAVISTA', 'BELLEVUE']
NE_3_2012 = ['GRAND ISLAND']

# Maine used the same districts from 2003-2013
ME_1_2012 = ['S. PORTLAND', 'AUGUSTA', 'SOUTH PORTLAND', 'PORTLAND', 'SCARBOROUGH', 'LEWISTON', 
             'KENNEBUNK', 'FREEPORT', 'WESTBROOK', 'FARMINGDALE', 'WATERVILLE', 'RICHMOND']
ME_2_2012 = ['BREWER', 'BETHEL', 'OXFORD', 'MACHIAS', 'HAMPDEN']

# 2016 
NE_1_2016 = [' LINCOLN', 'LINCOLN']
NE_2_2016 = ['OMAHA']
NE_3_2016 = ['SOUTH SIOUX CITY', 'HASTINGS']

ME_1_2016 = ['BIDDEFORD', 'RICHMOND', 'SCARBOROUGH', 'PORTLAND', 'SOUTH PORTLAND', 'AUGUSTA', 'SACO',
             'BATH', 'ROCKPORT', 'FALMOUTH', 'WELLS', 'LISBON', 'ROCKLAND']
ME_2_2016 = ['BANGOR', 'BELFAST', 'LEWISTON', 'ELLSWORTH']

# 2020 
NE_1_2020 = []
NE_2_2020 = ['OMAHA', 'PAPILLION']
NE_3_2020 = ['KEARNEY']

ME_1_2020 = ['AUGUSTA', 'PORTLAND', 'SACO', 'SCARBOROUGH', 'OLD ORCHARD BEACH', 'CAMDEN', 'BATH']
ME_2_2020 = ['LEWSITON', 'LEWISTON', 'BANGOR', 'PALMYRA', 'HERMON', 'ORRINGTON', 'NORTH ANSON', 'MACHIAS', 
             'HOLDEN', 'SOUTH PARIS', 'BAR HARBOR', 'ELLSWORTH', 'PRESQUE ISLE', 'SOUTHWEST HARBOR',
             'ELLSWORTH', 'STONINGTON', 'BREWER', 'FORT FAIRFIELD']

# 2024
NE_1_2024 = []
NE_2_2024 = ['OMAHA', 'BOYS TOWN']
NE_3_2024 = ['HASTINGS']

ME_1_2024 = ['SCARBOROUGH', 'LEBANON', 'PORTLAND', 'YORK BEACH']
ME_2_2024 = ['BANGOR']



Now that locations are separated by district we can add the new values

In [267]:
# R_2008
R_2008_NE_new = party_spending_dfs['2008_R'][party_spending_dfs['2008_R']['recipient_st'] == 'NE'].copy()
R_2008_NE_new.loc[R_2008_NE_new['recipient_city'].isin(NE_1_2008), 'recipient_st'] = 'NE-1'
R_2008_NE_new.loc[R_2008_NE_new['recipient_city'].isin(NE_2_2008), 'recipient_st'] = 'NE-2'
R_2008_NE_new.loc[R_2008_NE_new['recipient_city'].isin(NE_3_2008), 'recipient_st'] = 'NE-3'
# R_2008_NE_new = None 

R_2008_ME_new = party_spending_dfs['2008_R'][party_spending_dfs['2008_R']['recipient_st'] == 'ME'].copy()
R_2008_ME_new.loc[R_2008_ME_new['recipient_city'].isin(ME_1_2008), 'recipient_st'] = 'ME-1'
R_2008_ME_new.loc[R_2008_ME_new['recipient_city'].isin(ME_2_2008), 'recipient_st'] = 'ME-2'
# R_2008_ME_new = None

# D_2008
D_2008_NE_new = party_spending_dfs['2008_D'][party_spending_dfs['2008_D']['recipient_st'] == 'NE'].copy()
D_2008_NE_new.loc[D_2008_NE_new['recipient_city'].isin(NE_1_2008), 'recipient_st'] = 'NE-1'
D_2008_NE_new.loc[D_2008_NE_new['recipient_city'].isin(NE_2_2008), 'recipient_st'] = 'NE-2'
D_2008_NE_new.loc[D_2008_NE_new['recipient_city'].isin(NE_3_2008), 'recipient_st'] = 'NE-3'
# D_2008_NE_new = None

D_2008_ME_new = party_spending_dfs['2008_D'][party_spending_dfs['2008_D']['recipient_st'] == 'ME'].copy()
D_2008_ME_new.loc[D_2008_ME_new['recipient_city'].isin(ME_1_2008), 'recipient_st'] = 'ME-1'
D_2008_ME_new.loc[D_2008_ME_new['recipient_city'].isin(ME_2_2008), 'recipient_st'] = 'ME-2'
# D_2008_ME_new = None

# R_2012
R_2012_NE_new = party_spending_dfs['2012_R'][party_spending_dfs['2012_R']['recipient_st'] == 'NE'].copy()
R_2012_NE_new.loc[R_2012_NE_new['recipient_city'].isin(NE_1_2012), 'recipient_st'] = 'NE-1'
R_2012_NE_new.loc[R_2012_NE_new['recipient_city'].isin(NE_2_2012), 'recipient_st'] = 'NE-2'
R_2012_NE_new.loc[R_2012_NE_new['recipient_city'].isin(NE_3_2012), 'recipient_st'] = 'NE-3'
# R_2012_NE_new = None 

R_2012_ME_new = party_spending_dfs['2012_R'][party_spending_dfs['2012_R']['recipient_st'] == 'ME'].copy()
R_2012_ME_new.loc[R_2012_ME_new['recipient_city'].isin(ME_1_2012), 'recipient_st'] = 'ME-1'
R_2012_ME_new.loc[R_2012_ME_new['recipient_city'].isin(ME_2_2012), 'recipient_st'] = 'ME-2'
# R_2012_ME_new = None

# D_2012
D_2012_NE_new = party_spending_dfs['2012_D'][party_spending_dfs['2012_D']['recipient_st'] == 'NE'].copy()
D_2012_NE_new.loc[D_2012_NE_new['recipient_city'].isin(NE_1_2012), 'recipient_st'] = 'NE-1'
D_2012_NE_new.loc[D_2012_NE_new['recipient_city'].isin(NE_2_2012), 'recipient_st'] = 'NE-2'
D_2012_NE_new.loc[D_2012_NE_new['recipient_city'].isin(NE_3_2012), 'recipient_st'] = 'NE-3'
# D_2012_NE_new = None

D_2012_ME_new = party_spending_dfs['2012_D'][party_spending_dfs['2012_D']['recipient_st'] == 'ME'].copy()
D_2012_ME_new.loc[D_2012_ME_new['recipient_city'].isin(ME_1_2012), 'recipient_st'] = 'ME-1'
D_2012_ME_new.loc[D_2012_ME_new['recipient_city'].isin(ME_2_2012), 'recipient_st'] = 'ME-2'
# D_2012_ME_new = None

# R_2016
R_2016_NE_new = party_spending_dfs['2016_R'][party_spending_dfs['2016_R']['recipient_st'] == 'NE'].copy()
R_2016_NE_new.loc[R_2016_NE_new['recipient_city'].isin(NE_1_2016), 'recipient_st'] = 'NE-1'
R_2016_NE_new.loc[R_2016_NE_new['recipient_city'].isin(NE_2_2016), 'recipient_st'] = 'NE-2'
R_2016_NE_new.loc[R_2016_NE_new['recipient_city'].isin(NE_3_2016), 'recipient_st'] = 'NE-3'
# R_2016_NE_new = None 

R_2016_ME_new = party_spending_dfs['2016_R'][party_spending_dfs['2016_R']['recipient_st'] == 'ME'].copy()
R_2016_ME_new.loc[R_2016_ME_new['recipient_city'].isin(ME_1_2016), 'recipient_st'] = 'ME-1'
R_2016_ME_new.loc[R_2016_ME_new['recipient_city'].isin(ME_2_2016), 'recipient_st'] = 'ME-2'
# R_2016_ME_new = None

# D_2016
D_2016_NE_new = party_spending_dfs['2016_D'][party_spending_dfs['2016_D']['recipient_st'] == 'NE'].copy()
D_2016_NE_new.loc[D_2016_NE_new['recipient_city'].isin(NE_1_2016), 'recipient_st'] = 'NE-1'
D_2016_NE_new.loc[D_2016_NE_new['recipient_city'].isin(NE_2_2016), 'recipient_st'] = 'NE-2'
D_2016_NE_new.loc[D_2016_NE_new['recipient_city'].isin(NE_3_2016), 'recipient_st'] = 'NE-3'
# D_2016_NE_new = None

D_2016_ME_new = party_spending_dfs['2016_D'][party_spending_dfs['2016_D']['recipient_st'] == 'ME'].copy()
D_2016_ME_new.loc[D_2016_ME_new['recipient_city'].isin(ME_1_2016), 'recipient_st'] = 'ME-1'
D_2016_ME_new.loc[D_2016_ME_new['recipient_city'].isin(ME_2_2016), 'recipient_st'] = 'ME-2'
# D_2016_ME_new = None

# R_2020
R_2020_NE_new = party_spending_dfs['2020_R'][party_spending_dfs['2020_R']['recipient_st'] == 'NE'].copy()
R_2020_NE_new.loc[R_2020_NE_new['recipient_city'].isin(NE_1_2020), 'recipient_st'] = 'NE-1'
R_2020_NE_new.loc[R_2020_NE_new['recipient_city'].isin(NE_2_2020), 'recipient_st'] = 'NE-2'
R_2020_NE_new.loc[R_2020_NE_new['recipient_city'].isin(NE_3_2020), 'recipient_st'] = 'NE-3'
# R_2020_NE_new = None 

R_2020_ME_new = party_spending_dfs['2020_R'][party_spending_dfs['2020_R']['recipient_st'] == 'ME'].copy()
R_2020_ME_new.loc[R_2020_ME_new['recipient_city'].isin(ME_1_2020), 'recipient_st'] = 'ME-1'
R_2020_ME_new.loc[R_2020_ME_new['recipient_city'].isin(ME_2_2020), 'recipient_st'] = 'ME-2'
# R_2020_ME_new = None

# D_2020
D_2020_NE_new = party_spending_dfs['2020_D'][party_spending_dfs['2020_D']['recipient_st'] == 'NE'].copy()
D_2020_NE_new.loc[D_2020_NE_new['recipient_city'].isin(NE_1_2020), 'recipient_st'] = 'NE-1'
D_2020_NE_new.loc[D_2020_NE_new['recipient_city'].isin(NE_2_2020), 'recipient_st'] = 'NE-2'
D_2020_NE_new.loc[D_2020_NE_new['recipient_city'].isin(NE_3_2020), 'recipient_st'] = 'NE-3'
# D_2024_NE_new = None

D_2020_ME_new = party_spending_dfs['2020_D'][party_spending_dfs['2020_D']['recipient_st'] == 'ME'].copy()
D_2020_ME_new.loc[D_2020_ME_new['recipient_city'].isin(ME_1_2020), 'recipient_st'] = 'ME-1'
D_2020_ME_new.loc[D_2020_ME_new['recipient_city'].isin(ME_2_2020), 'recipient_st'] = 'ME-2'
# D_2024_ME_new = None


# R_2024
R_2024_NE_new = party_spending_dfs['2024_R'][party_spending_dfs['2024_R']['recipient_st'] == 'NE'].copy()
R_2024_NE_new.loc[R_2024_NE_new['recipient_city'].isin(NE_1_2024), 'recipient_st'] = 'NE-1'
R_2024_NE_new.loc[R_2024_NE_new['recipient_city'].isin(NE_2_2024), 'recipient_st'] = 'NE-2'
R_2024_NE_new.loc[R_2024_NE_new['recipient_city'].isin(NE_3_2024), 'recipient_st'] = 'NE-3'
# R_2024_NE_new = None 

R_2024_ME_new = party_spending_dfs['2024_R'][party_spending_dfs['2024_R']['recipient_st'] == 'ME'].copy()
R_2024_ME_new.loc[R_2024_ME_new['recipient_city'].isin(ME_1_2024), 'recipient_st'] = 'ME-1'
R_2024_ME_new.loc[R_2024_ME_new['recipient_city'].isin(ME_2_2024), 'recipient_st'] = 'ME-2'
# R_2024_ME_new = None

# D_2024
D_2024_NE_new = party_spending_dfs['2024_D'][party_spending_dfs['2024_D']['recipient_st'] == 'NE'].copy()
D_2024_NE_new.loc[D_2024_NE_new['recipient_city'].isin(NE_1_2024), 'recipient_st'] = 'NE-1'
D_2024_NE_new.loc[D_2024_NE_new['recipient_city'].isin(NE_2_2024), 'recipient_st'] = 'NE-2'
D_2024_NE_new.loc[D_2024_NE_new['recipient_city'].isin(NE_3_2024), 'recipient_st'] = 'NE-3'
# D_2024_NE_new = None

D_2024_ME_new = party_spending_dfs['2024_D'][party_spending_dfs['2024_D']['recipient_st'] == 'ME'].copy()
D_2024_ME_new.loc[D_2024_ME_new['recipient_city'].isin(ME_1_2024), 'recipient_st'] = 'ME-1'
D_2024_ME_new.loc[D_2024_ME_new['recipient_city'].isin(ME_2_2024), 'recipient_st'] = 'ME-2'
# D_2024_ME_new = None


In [268]:
party_spending_dfs['2008_R'] = pd.concat([party_spending_dfs['2008_R'], R_2008_NE_new, R_2008_ME_new])
party_spending_dfs['2012_R'] = pd.concat([party_spending_dfs['2012_R'], R_2012_NE_new, R_2012_ME_new])
party_spending_dfs['2016_R'] = pd.concat([party_spending_dfs['2016_R'], R_2016_NE_new, R_2016_ME_new])
party_spending_dfs['2020_R'] = pd.concat([party_spending_dfs['2020_R'], R_2020_NE_new, R_2020_ME_new])
party_spending_dfs['2024_R'] = pd.concat([party_spending_dfs['2024_R'], R_2024_NE_new, R_2024_ME_new])

party_spending_dfs['2008_D'] = pd.concat([party_spending_dfs['2008_D'], D_2008_NE_new, R_2008_ME_new])
party_spending_dfs['2012_D'] = pd.concat([party_spending_dfs['2012_D'], D_2012_NE_new, R_2012_ME_new])
party_spending_dfs['2016_D'] = pd.concat([party_spending_dfs['2016_D'], D_2016_NE_new, R_2016_ME_new])
party_spending_dfs['2020_D'] = pd.concat([party_spending_dfs['2020_D'], D_2020_NE_new, R_2020_ME_new])
party_spending_dfs['2024_D'] = pd.concat([party_spending_dfs['2024_D'], D_2024_NE_new, R_2024_ME_new])

# party_spending_dfs['2008_R']['recipient_st'].unique()
# for year,df in party_spending_dfs.items():
#     print(year, len(df['recipient_st'].unique()))


In [269]:
states.extend(['NE-1', 'NE-2', 'NE-3', 'ME-1', 'ME-2'])
# states.sort()
# states

['AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'FL',
 'GA',
 'IA',
 'IL',
 'IN',
 'LA',
 'MA',
 'MD',
 'ME',
 'MN',
 'MO',
 'NC',
 'NE',
 'NH',
 'NJ',
 'NY',
 'OK',
 'PA',
 'SC',
 'TN',
 'TX',
 'VA',
 'SD',
 'WA',
 'WI',
 'WY',
 'OH',
 'WV',
 'AK',
 'DE',
 'HI',
 'ID',
 'KS',
 'KY',
 'MI',
 'MS',
 'MT',
 'ND',
 'NM',
 'NV',
 'OR',
 'RI',
 'UT',
 'VT',
 'NE-1',
 'NE-2',
 'NE-3',
 'ME-1',
 'ME-2']

At the end we check to see if any states/DC are missing data, and see that several are missing entries. 

In [278]:
for year in party_spending_dfs:
    print(year, len(party_spending_dfs[year]['recipient_st'].unique()), 56-len(party_spending_dfs[year]['recipient_st'].unique()))

2008_R 55 1
2008_D 56 0
2012_R 56 0
2012_D 55 1
2016_R 55 1
2016_D 56 0
2020_R 54 2
2020_D 55 1
2024_R 52 4
2024_D 53 3


We then check to see if any states are missing. We see that several are, so we add them to a list as a tuple of the state missing and the year_party. 

In [280]:
# print(spending_dfs['2024']['recipient_st'].unique())
# print(party_spending_dfs['2024_R'][party_spending_dfs['2024_R']['recipient_st'] == 'SD'])

missed_states = []

for year, df in party_spending_dfs.items():
    for state in states:
        if state not in df['recipient_st'].unique():
            print(year, state)
            missed_states.append((year, state))

2008_R NE-3
2012_D NE-3
2016_R NE-3
2020_R NE-1
2020_R NE-3
2020_D NE-1
2024_R SD
2024_R NE-1
2024_R NE-3
2024_R ME-2
2024_D SD
2024_D NE-1
2024_D ME-2


We now calculate the total spend in each state by each candiate. We then tag it with the state and year (year is in the form {year}_{R|D}, R or D for republican or democrat) for compiling it all into one dataframe. 

In [281]:
party_years = list(party_spending_dfs.keys())

for year in party_years:
    # print(len(spending_dfs[year]['recipient_st'].unique()))
    state_sums[year] = party_spending_dfs[year].groupby('recipient_st').sum('disb_amt').sort_values('disb_amt')
    state_sums[year] = state_sums[year].drop('file_num', axis=1)


for year, st_sum in state_sums.items():
    st_sum['state_year'] = st_sum.index + f"_{year}"


Next add the missing states as 0 dollars spent. 

In [284]:

for year,state in missed_states:
    state_sums[year].loc[state]= [0, f'{state}_{year}']


We then combine all dataframes to get the total spend in each state per candidate for each year, we see that there are 560 entires, which matches 10 candidates and 56 states + DC + districts in Maine and Nebraska. 

In [283]:
all_state_sums_df = pd.concat(state_sums.values())
all_state_sums_df

,disb_amt,state_year
recipient_st,,
ND,2.982530e+03,ND_2008_R
WY,4.573600e+03,WY_2008_R
ID,6.016700e+03,ID_2008_R
HI,6.673290e+03,HI_2008_R
DE,6.685440e+03,DE_2008_R
...,...,...
GA,1.399371e+08,GA_2024_D
DC,2.110289e+08,DC_2024_D
SD,0.000000e+00,SD_2024_D
